In [ ]:
print('Ready for inference')

#### Since Internet is disabled for inference, I have cloned **ObjectDetectionfastai** library and saved in kaggle datasets.
* Install just from there
* Since we are just making predictions we are good to go without a **GPU**.


In [ ]:
!pip install ../input/objectdetectionfastai/ObjectDetection-master/ObjectDetection-master

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import defaultdict
import os
from fastai import *
from fastai.vision import *
from fastai.callbacks import *
import seaborn as sns 
import matplotlib.pyplot as plt
import matplotlib.image as immg
from sklearn.model_selection import StratifiedKFold,KFold

from object_detection_fastai.helper.object_detection_helper import *
from object_detection_fastai.loss.RetinaNetFocalLoss import RetinaNetFocalLoss
from object_detection_fastai.models.RetinaNet import RetinaNet
from object_detection_fastai.callbacks.callbacks import BBLossMetrics, BBMetrics, PascalVOCMetric

In [2]:
import warnings
warnings.filterwarnings("ignore",)

In [4]:
sns.set_style('darkgrid')

In [5]:
path = Path('/kaggle/input/global-wheat-detection');path.ls()

[PosixPath('/kaggle/input/global-wheat-detection/train.csv'),
 PosixPath('/kaggle/input/global-wheat-detection/train'),
 PosixPath('/kaggle/input/global-wheat-detection/test'),
 PosixPath('/kaggle/input/global-wheat-detection/sample_submission.csv')]

In [6]:
train = pd.read_csv(path/'train.csv')

In [7]:
tr = train.image_id.value_counts()
tr = pd.DataFrame({'image_id':tr.index,'wheat_count':tr.values})
tr = tr.sample(frac=1.,random_state=2020).reset_index(drop=True)

In [8]:
sample = pd.read_csv('../input/global-wheat-detection/sample_submission.csv')

In [9]:
def get_lbl_img(train):
    wheat2bbox = {}
    train['label'] = 'wheat'
    grp = train.image_id.unique()
    tr_gr = train.groupby(['image_id'])
    from tqdm.notebook import tqdm
    for i in tqdm(range(len(grp))):
        name = str(grp[i]) + '.jpg'
        bbox = []
        temp_b = []
        temp = tr_gr.get_group(grp[i])
        tt = temp.loc[:,'bbox'].values
        for j in range(len(temp)):
            t = tt[j][1:-1].split(',')
            t = [float(x) for x in t]  # x,y, width, height
            # Currently our coordinates are x,y,w,h and we want x1,y1,x2,y2
            # To convert it, we need to add our width and height to the respective x and y.
            t[2],t[3] = t[0]+t[2],t[1]+t[3]  
            # To achieve x2,y2 those we simply add width to x and height to y :
            # x2 = x + w and y2 = y + h
            t1 = [t[1],t[0],t[3],t[2]]   # inverse in fromat w,h for fastai
            temp_b.append(t1)
        bbox.append(temp_b)
        bbox.append(['wheat']*len(temp))
        wheat2bbox[name] = bbox
    return wheat2bbox

In [10]:
wheat2bbox = get_lbl_img(train)

## Test DataBunch

In [11]:
get_y_func = lambda o: wheat2bbox[Path(o).name] 

In [12]:
ts = (ObjectItemList.from_df(sample,path, folder = 'test' , suffix = '.jpg',cols='image_id'))

In [13]:
data = (ObjectItemList.from_df(tr,path, folder = 'train' , suffix = '.jpg',cols='image_id')
        #Where are the images? ->
        .split_by_rand_pct(0.2)                          
        #How to split in train/valid? -> randomly with the default 20% in valid
        .label_from_func(get_y_func)
        #How to find the labels? -> use get_y_func on the file name of the data
        .transform(size=512)
        .add_test(ts)
        #Data augmentation? -> Standard transforms; also transform the label images
        .databunch(bs=2, collate_fn=bb_pad_collate))   

In [14]:
size = 512

In [15]:
anchors = create_anchors(sizes=[(32,32),(16,16),(8,8),(4,4)], ratios=[0.5, 1, 2], scales=[0.35, 0.55, 0.75, 1, 1.25, 1.45])

In [16]:
model_dir = Path('../input/fastai-v1-global-wheat-detection-tutorial')

In [17]:
learn = load_learner(model_dir,'gwheat.pkl',test=ts)

ModuleNotFoundError: No module named 'object_detection_fastai'

In [ ]:
image_id = sample.image_id.values

In [ ]:
def show_output(item,bboxs_tot,scores_tot):
    fig,ax = plt.subplots(figsize=(10,10))
    ax.imshow(image2np(item.data))
    plt.axis('off')
    area_max = 512**2/5 
    for bbox, c in zip(bboxs_tot[0], scores_tot[0].numpy()):
        txt = 'wheat, {0:.4f}'.format(c)
        if bbox[2]*bbox[3] <= area_max:
            draw_rect(ax, [bbox[1],bbox[0],bbox[3],bbox[2]], text=txt,text_size=12,color='red')

In [ ]:
def process_preds_show(item,clas,bboxs,show_img,cnt,i):
    detect_thresh=0.4   # set your own detection threshold
    nms_thresh=0.1
    pred_string = []
    scores_tot = []
    bboxs_tot = []
    show_img = True if i<cnt else False
    for clas_pred, bbox_pred in list(zip(clas, bboxs)):
        bbox_pred, scores, preds = process_output(clas_pred, bbox_pred, anchors, detect_thresh)
        if bbox_pred is not None:
            to_keep = nms(bbox_pred, scores, nms_thresh)
            bbox_pred, preds, scores = bbox_pred[to_keep].cpu(), preds[to_keep].cpu(), scores[to_keep].cpu()
        t_sz = torch.Tensor([size])[None].cpu()
        if bbox_pred is not None:
            bbox_pred = to_np(rescale_boxes(bbox_pred, t_sz))
                # change from center to top left
            bbox_pred[:, :2] = bbox_pred[:, :2] - bbox_pred[:, 2:] / 2
            bboxs_tot.append(bbox_pred)
            scores_tot.append(scores)
    if show_img:
        show_output(item,bboxs_tot,scores_tot)
    area_max = (1024**2)/5
    for s,bbx in zip(scores_tot[0].numpy(),bboxs_tot[0]):
        bbx = [int(round(x)) for x in bbx*2]
        if bbx[2]*bbx[3] <= area_max :
            res = "{0:.4f} {1} {2} {3} {4}".format(s,bbx[1],bbx[0],bbx[3],bbx[2])
            pred_string.append(res)
    return pred_string

In [ ]:
def get_prediction(show_img=True,cnt=10): 
    # Set show img True to see img or else false for bboxs only, cnt for number of images to show
    preds_str = {}
    for i in range(len(data.test_ds)):
        item = learn.data.test_ds[i][0]  #Pick one image
        batch = learn.data.one_item(item)
        clas,bboxs,xtr = learn.pred_batch(batch=batch)
        prd = process_preds_show(item,clas,bboxs,show_img,cnt,i) 
        preds_str[image_id[i]] = " ".join(prd)
    return preds_str

* A look at first test images

In [ ]:
prediction = get_prediction(cnt=0)
# Set False to not show images
# Regardless of that it will give prediction string 

In [ ]:
submit = pd.DataFrame.from_dict(prediction,orient='index').reset_index()
submit.columns = ['image_id','PredictionString']
submit.head(10)

In [ ]:
submit.to_csv('submission.csv',index=False)